In [1]:
import requests
import json
import pandas as pd
from pandas import json_normalize

In [2]:
# API URL'si
url = "https://ws.cso.ie/public/api.restful/PxStat.Data.Cube_API.ReadDataset/ROA26/JSON-stat/1.0/en"

# API anahtarınız
#api_key = "21595794ce41476d813453e51ee0b17e"

In [ ]:
#data = json.loads(res.text)
#data

In [3]:
# API'den veri çekmek için GET isteği yapılıyor
res = requests.get(url)

# Eğer istek başarılıysa (HTTP 200), veriyi al
if res.status_code == 200:
    data = res.json()  # JSON formatındaki veriyi Python sözlüğüne dönüştürür
    print("Response=",res.status_code,"\n")
    print(data)
else:
    print("API'den veri çekilemedi. Hata kodu:", response.status_code)


Response= 200 

{'dataset': {'dimension': {'STATISTIC': {'label': 'Statistic', 'category': {'index': {'ROA26C01': 0, 'ROA26C02': 1}, 'label': {'ROA26C01': 'Full Driving Licences', 'ROA26C02': 'Learner Permit Driving Licences'}, 'unit': {'ROA26C01': {'base': 'Number'}, 'ROA26C02': {'base': 'Number'}}}}, 'TLIST(A1)': {'label': 'Year', 'category': {'index': {'2013': 0, '2014': 1, '2015': 2, '2016': 3, '2017': 4, '2018': 5, '2019': 6, '2020': 7, '2021': 8, '2022': 9}, 'label': {'2013': '2013', '2014': '2014', '2015': '2015', '2016': '2016', '2017': '2017', '2018': '2018', '2019': '2019', '2020': '2020', '2021': '2021', '2022': '2022'}}}, 'C02076V03371': {'label': 'Age Group', 'category': {'index': {'-': 0, '336': 1, '3401': 2, '385': 3, '410': 4, '445': 5, '480': 6, '520': 7, '555': 8, '585': 9, '620': 10}, 'label': {'-': 'All ages', '336': 'Under 17 years', '3401': '17 - 20 years', '385': '21 - 24 years', '410': '25 - 29 years', '445': '30 - 39 years', '480': '40 - 49 years', '520': '50 -

In [4]:
data.keys()

dict_keys(['dataset'])

In [5]:
data['dataset']

{'dimension': {'STATISTIC': {'label': 'Statistic',
   'category': {'index': {'ROA26C01': 0, 'ROA26C02': 1},
    'label': {'ROA26C01': 'Full Driving Licences',
     'ROA26C02': 'Learner Permit Driving Licences'},
    'unit': {'ROA26C01': {'base': 'Number'}, 'ROA26C02': {'base': 'Number'}}}},
  'TLIST(A1)': {'label': 'Year',
   'category': {'index': {'2013': 0,
     '2014': 1,
     '2015': 2,
     '2016': 3,
     '2017': 4,
     '2018': 5,
     '2019': 6,
     '2020': 7,
     '2021': 8,
     '2022': 9},
    'label': {'2013': '2013',
     '2014': '2014',
     '2015': '2015',
     '2016': '2016',
     '2017': '2017',
     '2018': '2018',
     '2019': '2019',
     '2020': '2020',
     '2021': '2021',
     '2022': '2022'}}},
  'C02076V03371': {'label': 'Age Group',
   'category': {'index': {'-': 0,
     '336': 1,
     '3401': 2,
     '385': 3,
     '410': 4,
     '445': 5,
     '480': 6,
     '520': 7,
     '555': 8,
     '585': 9,
     '620': 10},
    'label': {'-': 'All ages',
     '336': 

In [6]:
data['dataset'].keys()

dict_keys(['dimension', 'label', 'source', 'updated', 'value'])

In [7]:
data['dataset']['label']

'Current Driving Licences'

In [8]:
data['dataset']['dimension'].keys()

dict_keys(['STATISTIC', 'TLIST(A1)', 'C02076V03371', 'C03424V04122', 'role', 'id', 'size'])

In [9]:
data['dataset']['dimension']['TLIST(A1)']

{'label': 'Year',
 'category': {'index': {'2013': 0,
   '2014': 1,
   '2015': 2,
   '2016': 3,
   '2017': 4,
   '2018': 5,
   '2019': 6,
   '2020': 7,
   '2021': 8,
   '2022': 9},
  'label': {'2013': '2013',
   '2014': '2014',
   '2015': '2015',
   '2016': '2016',
   '2017': '2017',
   '2018': '2018',
   '2019': '2019',
   '2020': '2020',
   '2021': '2021',
   '2022': '2022'}}}

In [10]:
df = pd.DataFrame(data)
df

,dataset
dimension,"{'STATISTIC': {'label': 'Statistic', 'category..."
label,Current Driving Licences
source,Road Safety Authority (RSA)
updated,2023-11-22T11:00:00Z
value,"[2414393.0, 32543.0, 585835.0, 110204.0, 50544..."


In [11]:
# İç içe geçmiş JSON verisini düzleştirmek için json_normalize kullanın
df = json_normalize(data)
df

,dataset.dimension.STATISTIC.label,dataset.dimension.STATISTIC.category.index.ROA26C01,dataset.dimension.STATISTIC.category.index.ROA26C02,dataset.dimension.STATISTIC.category.label.ROA26C01,dataset.dimension.STATISTIC.category.label.ROA26C02,dataset.dimension.STATISTIC.category.unit.ROA26C01.base,dataset.dimension.STATISTIC.category.unit.ROA26C02.base,dataset.dimension.TLIST(A1).label,dataset.dimension.TLIST(A1).category.index.2013,dataset.dimension.TLIST(A1).category.index.2014,...,dataset.dimension.C03424V04122.category.label.25,dataset.dimension.C03424V04122.category.label.26,dataset.dimension.role.metric,dataset.dimension.role.time,dataset.dimension.id,dataset.dimension.size,dataset.label,dataset.source,dataset.updated,dataset.value
0,Statistic,0,1,Full Driving Licences,Learner Permit Driving Licences,Number,Number,Year,0,1,...,Donegal,Monaghan,[STATISTIC],[TLIST(A1)],"[STATISTIC, TLIST(A1), C02076V03371, C03424V04...","[2, 10, 11, 27]",Current Driving Licences,Road Safety Authority (RSA),2023-11-22T11:00:00Z,"[2414393.0, 32543.0, 585835.0, 110204.0, 50544..."


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Columns: 114 entries, dataset.dimension.STATISTIC.label to dataset.value
dtypes: int64(50), object(64)
memory usage: 1.0+ KB


In [13]:
df = pd.read_json(url)
print(df)


                                                     dataset
dimension  {'STATISTIC': {'label': 'Statistic', 'category...
label                               Current Driving Licences
source                           Road Safety Authority (RSA)
updated                                 2023-11-22T11:00:00Z
value      [2414393.0, 32543.0, 585835.0, 110204.0, 50544...


In [15]:
df = pd.DataFrame({
    "Time" : data['dataset']['dimension'],
    "bolum" : data['dataset']['dimension']['TLIST(A1)']
})
df

,Time,bolum
STATISTIC,"{'label': 'Statistic', 'category': {'index': {...",NaN
TLIST(A1),"{'label': 'Year', 'category': {'index': {'2013...",NaN
C02076V03371,"{'label': 'Age Group', 'category': {'index': {...",NaN
C03424V04122,"{'label': 'Licensing Authority', 'category': {...",NaN
role,"{'metric': ['STATISTIC'], 'time': ['TLIST(A1)']}",NaN
id,"[STATISTIC, TLIST(A1), C02076V03371, C03424V04...",NaN
size,"[2, 10, 11, 27]",NaN
label,NaN,Year
category,NaN,"{'index': {'2013': 0, '2014': 1, '2015': 2, '2..."


In [16]:
data['dataset']['dimension']['TLIST(A1)']['category']

{'index': {'2013': 0,
  '2014': 1,
  '2015': 2,
  '2016': 3,
  '2017': 4,
  '2018': 5,
  '2019': 6,
  '2020': 7,
  '2021': 8,
  '2022': 9},
 'label': {'2013': '2013',
  '2014': '2014',
  '2015': '2015',
  '2016': '2016',
  '2017': '2017',
  '2018': '2018',
  '2019': '2019',
  '2020': '2020',
  '2021': '2021',
  '2022': '2022'}}